<a href="https://colab.research.google.com/github/PyAshishMhatre/DigitalMarketing-Algorithms-Project/blob/main/Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet -U implicit
!pip install unidecode
!pip install --quiet -U pinecone-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import time
import random
import numpy as np
import pandas as pd
import scipy.sparse as sparse
import itertools
from unidecode import unidecode
import pinecone
import implicit
from implicit import evaluation

# Data Prep

In [ ]:
#Reading Order_items and products file
orders_df = pd.read_csv(r"order_items.csv")
products = pd.read_csv(r"products.csv")
users_df = pd.read_csv(r"users.csv")

In [ ]:
orders = pd.merge(orders_df, users_df, left_on='user_id', right_on='id', how='inner')

In [ ]:
orders.head()

,id_x,order_id,user_id,product_id,inventory_item_id,status,created_at_x,shipped_at,delivered_at,returned_at,...,gender,state,street_address,postal_code,city,country,latitude,longitude,traffic_source,created_at_y
0,82431,56667,45224,13606,222767,Returned,2022-08-07 06:27:15 UTC,2022-08-08 06:10:00 UTC,2022-08-12 12:50:00 UTC,2022-08-13 19:51:00 UTC,...,F,England,24374 Solis Gardens Apt. 084,NG6,Nottingham,United Kingdom,53.009832,-1.187026,Search,2019-12-24 07:03:00 UTC
1,82433,56668,45224,2122,222772,Complete,2020-07-11 05:40:44 UTC,2020-07-13 21:10:00 UTC,2020-07-16 23:08:00 UTC,NaN,...,F,England,24374 Solis Gardens Apt. 084,NG6,Nottingham,United Kingdom,53.009832,-1.187026,Search,2019-12-24 07:03:00 UTC
2,82430,56667,45224,5773,222765,Returned,2022-08-09 05:32:50 UTC,2022-08-08 06:10:00 UTC,2022-08-12 12:50:00 UTC,2022-08-13 19:51:00 UTC,...,F,England,24374 Solis Gardens Apt. 084,NG6,Nottingham,United Kingdom,53.009832,-1.187026,Search,2019-12-24 07:03:00 UTC
3,82432,56667,45224,11461,222770,Returned,2022-08-08 05:04:41 UTC,2022-08-08 06:10:00 UTC,2022-08-12 12:50:00 UTC,2022-08-13 19:51:00 UTC,...,F,England,24374 Solis Gardens Apt. 084,NG6,Nottingham,United Kingdom,53.009832,-1.187026,Search,2019-12-24 07:03:00 UTC
4,76930,52883,42129,28951,207892,Shipped,2022-12-31 05:59:28 UTC,2022-12-31 00:13:00 UTC,NaN,NaN,...,M,Guangdong,0681 Krista Villages,518042,Shenzhen,China,22.530887,114.036907,Search,2020-04-24 08:19:00 UTC


In [ ]:
#Product Data formatting
products = products.dropna(subset=['name'])
products = products.rename(columns={'id': 'product_id', 'name': 'product_name'})
products = products[['product_id', 'category','product_name', 'brand', 'department']]

#Aggregating Orders data creating confidence column "Quantity"

orders = orders.groupby(['city', 'product_id']).size().reset_index(name="Quantity")
data = orders.copy()

In [ ]:
products.head()

,product_id,category,product_name,brand,department
0,27569,Swim,2XU Men's Swimmers Compression Long Sleeve Top,2XU,Men
1,27445,Swim,TYR Sport Men's Square Leg Short Swim Suit,TYR,Men
2,27457,Swim,TYR Sport Men's Solid Durafast Jammer Swim Suit,TYR,Men
3,27466,Swim,TYR Sport Men's Swim Short/Resistance Short Sw...,TYR,Men
4,27481,Swim,TYR Alliance Team Splice Jammer,TYR,Men


In [ ]:
orders.head()

,city,product_id,Quantity
0,Aachen,3210,1
1,Aachen,3221,1
2,Aachen,4214,1
3,Aachen,5438,1
4,Aachen,6361,1


In [ ]:
data.head()

,city,product_id,Quantity
0,Aachen,3210,1
1,Aachen,3221,1
2,Aachen,4214,1
3,Aachen,5438,1
4,Aachen,6361,1


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

def convert_categorical_to_numerical(df):
    """
    Converts all categorical columns in a pandas DataFrame to numerical values using Scikit-Learn's LabelEncoder.
    
    Parameters:
    -----------
    df : pandas DataFrame
        The DataFrame containing categorical columns to be converted.
    
    Returns:
    --------
    pandas DataFrame
        The DataFrame with all categorical columns converted to numerical values.
    """
    le = LabelEncoder()
    df_new = df.copy()
    for column in df.select_dtypes(include=['object']):
        df_new[column] = le.fit_transform(df_new[column])
    
    return df_new


In [ ]:
data = convert_categorical_to_numerical(data)

In [ ]:
orders

,city,product_id,Quantity
0,Aachen,3210,1
1,Aachen,3221,1
2,Aachen,4214,1
3,Aachen,5438,1
4,Aachen,6361,1
...,...,...,...
178202,海 名,8812,1
178203,海 名,14712,1
178204,海 名,14845,1
178205,海 名,15463,1


In [ ]:
lookup_city = pd.merge(orders,data, left_index=True, right_index=True)
lookup_city = lookup_city[['city_x', 'city_y']].copy()

In [ ]:
lookup_city.head()

,city_x,city_y
0,Aachen,0
1,Aachen,0
2,Aachen,0
3,Aachen,0
4,Aachen,0


In [ ]:
data

,city,product_id,Quantity
0,0,3210,1
1,0,3221,1
2,0,4214,1
3,0,5438,1
4,0,6361,1
...,...,...,...
178202,7563,8812,1
178203,7563,14712,1
178204,7563,14845,1
178205,7563,15463,1


In [ ]:
print('Length of unique cities in data', data.city.nunique())
print('Length of unique products in product', products.product_id.nunique())

Length of unique cities in data 7564
Length of unique products in product 29118


In [ ]:
city = list(np.sort(data.city.unique()))
items = list(np.sort(products.product_id.unique()))
purchases = list(data.Quantity)

# create zero-based index position <-> user/item ID mappings
index_to_city = pd.Series(city)

# create reverse mappings from user/item ID to index positions
city_to_index = pd.Series(data=index_to_city.index + 1, index=index_to_city.values)

# create zero-based index position <-> item/user ID mappings
index_to_item = pd.Series(items)

# create reverse mapping from item/user ID to index positions
item_to_index = pd.Series(data=index_to_item.index, index=index_to_item.values)

# Get the rows and columns for our new matrix
products_rows = data.product_id.astype(int)
city_cols = data.city.astype(int)

# Create a sparse matrix for our users and products containing number of purchases

## sparse_product_user ==> sparse_product_city
## sparse_user_product ==> sparse_city_product

sparse_product_city = sparse.csr_matrix((purchases, (products_rows, city_cols)))
sparse_product_city.data = np.nan_to_num(sparse_product_city.data, copy=False)

sparse_city_product = sparse.csr_matrix((purchases, (city_cols, products_rows)))
sparse_city_product.data = np.nan_to_num(sparse_city_product.data, copy=False)

In [ ]:
sparse_product_city.shape

(29121, 7564)

# Model 

In [ ]:
#split data into train and test sets
train_set, test_set = evaluation.train_test_split(sparse_product_city, train_percentage=0.8)

# initialize a model
model = implicit.als.AlternatingLeastSquares(factors=100,
                                             regularization=0.05,
                                             iterations=20,
                                             num_threads=1)

alpha_val = 40
train_set = (train_set * alpha_val).astype('double')

# train the model on a sparse matrix of item/user/confidence weights
model.fit(train_set, show_progress = True)

  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
#model Evaluation

test_set = (test_set * alpha_val).astype('double')
evaluation.ranking_metrics_at_k(model, train_set, test_set, K=100,
                         show_progress=True, num_threads=1)

  0%|          | 0/20521 [00:00<?, ?it/s]

{'precision': 0.12064711489934392,
 'map': 0.012941282170683976,
 'ndcg': 0.03684403970128409,
 'auc': 0.5536280249841881}

In [ ]:
len(model.item_factors)

7564

In [ ]:
len(model.user_factors)

29121

# Pinecone setup

In [ ]:
pinecone.delete_index('products-recomm')

In [ ]:
pinecone.init(api_key='d9bee8c5-bb9e-4764-a3fd-e45519c5c566', environment='eu-west1-gcp')

# Set and create index  
index_name = 'products-recomm'
# Make sure service with the same name does not exist
if index_name in pinecone.list_indexes():
    index = pinecone.Index(index_name=index_name) 
else:
  pinecone.create_index(name=index_name, dimension=100)
                      

index = pinecone.Index(index_name=index_name) 

In [ ]:
#Making Data ready for Pinecone upsert

# Get all of the items
all_items = [title for title in products['product_name']]

# Replace non-ASCII characters in item IDs with ASCII equivalents
all_items_ascii = [unidecode(title) for title in all_items]

# Transform items into factors
user_factors = model.user_factors

# Prepare item factors for upload
items_to_insert = list(zip(all_items_ascii, user_factors[1:].tolist()))
# display(items_to_insert[:1])

In [ ]:
len(items_to_insert)

29118

In [ ]:
#chunks function

def chunks(iterable, batch_size=100):
    it = iter(iterable)
    chunk = tuple(itertools.islice(it, batch_size))
    while chunk:
        yield chunk
        chunk = tuple(itertools.islice(it, batch_size))

In [ ]:
#Insert into pinecone 
print('Index statistics before upsert:', index.describe_index_stats())

for e, batch in enumerate(chunks([(ii[:64],x) for ii,x in items_to_insert])):
    index.upsert(vectors=batch)

print('Index statistics after upsert:', index.describe_index_stats())

Index statistics before upsert: {'dimension': 100,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}
Index statistics after upsert: {'dimension': 100,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 27220}},
 'total_vector_count': 27220}


# Helper Function

In [ ]:
#helper function
def get_city(id):
    dff = data[data['city'] == id]
    return dff

def get_product(id):
    dff = data[data['product_id'] == id]
    return dff


def products_bought_by_city_in_the_past(id: int, top: int = 10):

    selected = data[data.city == id].sort_values(by=['Quantity'], ascending=False)

    selected['product_name'] = selected['product_id'].map(products.set_index('product_id')['product_name'])
    selected = selected[['product_id', 'product_name', 'Quantity']].reset_index(drop=True)
    selected['category'] = selected['product_id'].map(products.set_index('product_id')['category'])
    selected['department'] = selected['product_id'].map(products.set_index('product_id')['department'])
    if selected.shape[0] < top:
        return selected

    return selected[:top]

# get the column for product 6789
# value = user_item[99999, 10394]
# value


# Pinecone Recomm

In [ ]:
city_ids = [0,2,5]
item_factors = model.item_factors[city_to_index[city_ids]]

# display(user_factors[1:])

In [ ]:
# Query by user factors

start_time = time.process_time()
query_results = index.query(queries=item_factors[:-1].tolist(), top_k=10)
print("Time needed for retrieving recommended products using Pinecone: " + str(time.process_time() - start_time) + ' seconds.\n')

for _id, res in zip(city_ids, query_results.results):
    print(f'user_id={_id}')

    #recommendation 
    p = [match.id for match in res.matches]
    s = [match.score for match in res.matches]
    df = pd.DataFrame(
        {
            'product_name': p,
            'scores': s
        }
    )
    df = df.merge(products, on = 'product_name', how = 'inner')[['product_id','product_name', 'category','department','scores' ]]

    # Filter Recommendation
    # dep = list(products_bought_by_city_in_the_past(_id, top=15)['department'])[0]
    # df = df[df['department'] == dep]

    print("Recommendation: ")
    display(df)
    print("Top buys from the past: ")
    display(products_bought_by_city_in_the_past(_id, top=15))

Time needed for retrieving recommended products using Pinecone: 0.006856605999985277 seconds.

user_id=0
Recommendation: 


,product_id,product_name,category,department,scores
0,22487,Supercords / Pleated Front Dark Brown Cuffed,Pants,Men,0.745378
1,4830,Diesel Women's Livier Jean,Jeans,Women,0.598995
2,3172,maxandcleo Women's Whitney Striped Dress,Dresses,Women,0.594959
3,27434,Speedo Men's Keepin' On Short Sleeve Rashguard...,Swim,Men,0.585005
4,13175,Betsey Johnson Beauty Mark Halter Bra Top,Swim,Women,0.568843
5,9003,K. Bell Socks Women's Over The Knee Rouched Top,Socks & Hosiery,Women,0.565259
6,24124,Reebok Mens Full Zip Ripstop Lightweight Jacket,Outerwear & Coats,Men,0.555758
7,14233,Native Eyewear Hardtop XP Sunglasses,Accessories,Women,0.549557
8,3039,2XU Women's Compression Race Sock,Active,Women,0.545939
9,9041,2XU Women's Compression Race Sock,Socks & Hosiery,Women,0.545939


Top buys from the past: 


,product_id,product_name,Quantity,category,department
0,3210,KAMALIKULTURE Women's Long Sleeve Side Draped ...,1,Dresses,Women
1,24901,Smartwool PhD Nordic Ultra Light Socks,1,Socks,Men
2,22274,Dockers Men's Limited Offer Linen Drawstring Pant,1,Pants,Men
3,22286,CornerStone Elastic Insert Pant. PT60,1,Pants,Men
4,22804,Oakley Men's Represent Short,1,Shorts,Men
5,23878,River's End Fully Lined Microfiber Windshirt,1,Outerwear & Coats,Men
6,23931,Tommy Hilfiger Men's Wool Plush Peacoat,1,Outerwear & Coats,Men
7,24638,Hanes Classics Men's 6-Pack Cushion Low Cut Socks,1,Socks,Men
8,25096,Polo Ralph Lauren Cotton Argyle Socks 3 Pair S...,1,Socks,Men
9,20421,Brand New Landisun Wedding Tuxedo Silk Vest / ...,1,Suits & Sport Coats,Men


user_id=2
Recommendation: 


,product_id,product_name,category,department,scores
0,12248,3 PC. Ladies Cross BoneTube Top Set - One Size...,Intimates,Women,0.649126
1,11424,Mary Green Silk Knit 14 Inch Mini Half Slip (L...,Intimates,Women,0.593164
2,8525,Alfred Dunner Women's Spliced Sequin Jacket,Outerwear & Coats,Women,0.590150
3,3694,Anne Klein Women's Tuxedo Dress,Dresses,Women,0.512766
4,6,Wilt Women's Color Blocked Big Mixed Slant Top,Tops & Tees,Women,0.444251
5,14192,New Trendy Pixel Shape Wayfarer Sunglasses,Accessories,Women,0.395079
6,2635,Icebreaker Women's 260 Midweight Leggings,Active,Women,0.369374
7,15381,Dr. Rey Shapewear Womens Firm Control High Wai...,Plus,Women,0.353511
8,11091,Dr. Rey Shapewear Womens Firm Control High Wai...,Intimates,Women,0.353511
9,18933,Smartwool Capitol Peak V-Neck - Men's,Sweaters,Men,0.352284


Top buys from the past: 


,product_id,product_name,Quantity,category,department
0,1670,District Threads - Junior Ladies Long Sleeve T...,1,Fashion Hoodies & Sweatshirts,Women
1,1746,Womens Rihanna Lips Sweater Jumper Top,1,Fashion Hoodies & Sweatshirts,Women
2,3094,KEEN Women Bellingham Crew Lite Sock,1,Active,Women
3,3310,KAMALIKULTURE Women's Long Sleeve Side Draped ...,1,Dresses,Women
4,4735,7 For All Mankind Women's The Skinny Jean,1,Jeans,Women
5,5556,Red Valentino Womens Bow Stitch Front Pockets ...,1,Pants & Capris,Women
6,12841,Kanu Surf Women's Imagine Shirt,1,Swim,Women
7,13249,Roxy Women's Wave Wanderer Surfer Pant,1,Swim,Women
8,13641,Premium Soft Viscose Flower Print Scarf - Diff...,1,Accessories,Women
9,15287,Playtex Women's 18 Hour Seamless Smoothing Bra...,1,Plus,Women


# Recommendations based on Gender

In [244]:
#Reading Order_items and products file
orders_df = pd.read_csv(r"order_items.csv")
products = pd.read_csv(r"products.csv")
users_df = pd.read_csv(r"users.csv")

In [245]:
orders = pd.merge(orders_df, users_df, left_on='user_id', right_on='id', how='inner')

In [246]:
#Product Data formatting
products = products.dropna(subset=['name'])
products = products.rename(columns={'id': 'product_id', 'name': 'product_name'})
products = products[['product_id', 'category','product_name', 'brand', 'department']]

#Aggregating Orders data creating confidence column "Quantity"

orders = orders.groupby(['gender', 'product_id']).size().reset_index(name="Quantity")
data = orders.copy()

In [247]:
data = convert_categorical_to_numerical(data)

In [248]:
orders

,gender,product_id,Quantity
0,F,1,8
1,F,2,7
2,F,3,1
3,F,4,5
4,F,5,3
...,...,...,...
29052,M,29116,6
29053,M,29117,8
29054,M,29118,5
29055,M,29119,2


In [249]:
data

,gender,product_id,Quantity
0,0,1,8
1,0,2,7
2,0,3,1
3,0,4,5
4,0,5,3
...,...,...,...
29052,1,29116,6
29053,1,29117,8
29054,1,29118,5
29055,1,29119,2


In [250]:
print('Length of unique cities in data', data.gender.nunique())
print('Length of unique products in product', products.product_id.nunique())

Length of unique cities in data 2
Length of unique products in product 29118


In [251]:
gender = list(np.sort(data.gender.unique()))
items = list(np.sort(products.product_id.unique()))
purchases = list(data.Quantity)

# create zero-based index position <-> user/item ID mappings
index_to_gender = pd.Series(gender)

# create reverse mappings from user/item ID to index positions
gender_to_index = pd.Series(data=index_to_gender.index, index=index_to_gender.values)

# create zero-based index position <-> item/user ID mappings
index_to_item = pd.Series(items)

# create reverse mapping from item/user ID to index positions
item_to_index = pd.Series(data=index_to_item.index, index=index_to_item.values)

# Get the rows and columns for our new matrix
products_rows = data.product_id.astype(int)
gender_cols = data.gender.astype(int)

# Create a sparse matrix for our users and products containing number of purchases

## sparse_product_user ==> sparse_product_gender
## sparse_user_product ==> sparse_gender_product

sparse_product_gender = sparse.csr_matrix((purchases, (products_rows, gender_cols)))
sparse_product_gender.data = np.nan_to_num(sparse_product_gender.data, copy=False)

sparse_gender_product = sparse.csr_matrix((purchases, (gender_cols, products_rows)))
sparse_gender_product.data = np.nan_to_num(sparse_gender_product.data, copy=False)

In [252]:
sparse_product_gender.shape

(29121, 2)

In [253]:
#split data into train and test sets
train_set, test_set = evaluation.train_test_split(sparse_product_gender, train_percentage=0.8)

# initialize a model
model = implicit.als.AlternatingLeastSquares(factors=100,
                                             regularization=0.05,
                                             iterations=20,
                                             num_threads=1)

alpha_val = 40
train_set = (train_set * alpha_val).astype('double')

# train the model on a sparse matrix of item/user/confidence weights
model.fit(train_set, show_progress = True)

  0%|          | 0/20 [00:00<?, ?it/s]

In [254]:
#model Evaluation

test_set = (test_set * alpha_val).astype('double')
evaluation.ranking_metrics_at_k(model, train_set, test_set, K=100,
                         show_progress=True, num_threads=1)

  0%|          | 0/5789 [00:00<?, ?it/s]

{'precision': 54.7484885126965,
 'map': 52.45190471264528,
 'ndcg': 11.386989121755178,
 'auc': 0.45154603558472967}

In [255]:
print(model.user_factors.shape)
print(model.item_factors.shape)

(29121, 100)
(2, 100)


In [256]:
pinecone.delete_index('products-recomm')

In [257]:
pinecone.init(api_key='d9bee8c5-bb9e-4764-a3fd-e45519c5c566', environment='eu-west1-gcp')

# Set and create index  
index_name = 'products-recomm'
# Make sure service with the same name does not exist
if index_name in pinecone.list_indexes():
    index = pinecone.Index(index_name=index_name) 
else:
  pinecone.create_index(name=index_name, dimension=100)
                      

index = pinecone.Index(index_name=index_name) 

In [258]:
#Making Data ready for Pinecone upsert

# Get all of the items
all_items = [title for title in products['product_name']]

# Replace non-ASCII characters in item IDs with ASCII equivalents
all_items_ascii = [unidecode(title) for title in all_items]

# Transform items into factors
user_factors = model.user_factors

# Prepare item factors for upload
items_to_insert = list(zip(all_items_ascii, user_factors[:].tolist()))
# display(items_to_insert[:1])

In [259]:
len(items_to_insert)

29118

In [260]:
#chunks function

def chunks(iterable, batch_size=100):
    it = iter(iterable)
    chunk = tuple(itertools.islice(it, batch_size))
    while chunk:
        yield chunk
        chunk = tuple(itertools.islice(it, batch_size))

In [261]:
#Insert into pinecone 
print('Index statistics before upsert:', index.describe_index_stats())

for e, batch in enumerate(chunks([(ii[:64],x) for ii,x in items_to_insert])):
    index.upsert(vectors=batch)

print('Index statistics after upsert:', index.describe_index_stats())

Index statistics before upsert: {'dimension': 100,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}
Index statistics after upsert: {'dimension': 100,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 27220}},
 'total_vector_count': 27220}


In [262]:
#helper function
def get_gender(id):
    dff = data[data['gender'] == id]
    return dff

def get_product(id):
    dff = data[data['product_id'] == id]
    return dff


def products_bought_by_gender_in_the_past(id: int, top: int = 10):

    selected = data[data.gender == id].sort_values(by=['Quantity'], ascending=False)

    selected['product_name'] = selected['product_id'].map(products.set_index('product_id')['product_name'])
    selected = selected[['product_id', 'product_name', 'Quantity']].reset_index(drop=True)
    selected['category'] = selected['product_id'].map(products.set_index('product_id')['category'])
    selected['department'] = selected['product_id'].map(products.set_index('product_id')['department'])
    if selected.shape[0] < top:
        return selected

    return selected[:top]

# get the column for product 6789
# value = user_item[99999, 10394]
# value


In [263]:
len(model.item_factors)

2

In [264]:
gender_to_index

0    0
1    1
dtype: int64

In [270]:
gender_ids = [0]
item_factors = model.item_factors[gender_to_index[gender_ids]]

# display(user_factors[1:])

In [271]:
# Query by user factors

start_time = time.process_time()
query_results = index.query(queries=item_factors[:].tolist(), top_k=10)
print("Time needed for retrieving recommended products using Pinecone: " + str(time.process_time() - start_time) + ' seconds.\n')

for _id, res in zip(city_ids, query_results.results):
    print(f'user_id={_id}')

    #recommendation 
    p = [match.id for match in res.matches]
    s = [match.score for match in res.matches]
    df = pd.DataFrame(
        {
            'product_name': p,
            'scores': s
        }
    )
    df = df.merge(products, on = 'product_name', how = 'inner')[['product_id','product_name', 'category','department','scores' ]]

    # Filter Recommendation
    # dep = list(products_bought_by_city_in_the_past(_id, top=15)['department'])[0]
    # df = df[df['department'] == dep]

    print("Recommendation: ")
    display(df)
    print("Top buys from the past: ")
    display(products_bought_by_gender_in_the_past(_id, top=15))

Time needed for retrieving recommended products using Pinecone: 0.004007021000006716 seconds.

user_id=0
Recommendation: 


,product_id,product_name,category,department,scores
0,23408,COPPER COVE Side Slide Pleated Shorts #C3021,Shorts,Men,1.0
1,22898,Armani Exchange Zip Pocket Cargo Short,Shorts,Men,1.0
2,19392,ambiguous Men's Coleman Sweater,Sweaters,Men,1.0
3,19618,Paul Fredrick 100% Cotton Stripe Cardigan Sweater,Sweaters,Men,1.0
4,19169,Men's Traditional Merino Wool Blue Aran Irish ...,Sweaters,Men,1.0
5,19164,Original Penguin Men's Kris Sweater,Sweaters,Men,1.0
6,28046,parke & ronen Men's Stretch Microfiber Angelen...,Swim,Men,1.0
7,18835,Calvin Klein Sportswear Men's Transitional Cot...,Sweaters,Men,1.0
8,18882,Dockers Men's Buffalo Plaid Crew Sweater,Sweaters,Men,1.0


Top buys from the past: 


,product_id,product_name,Quantity,category,department
0,396,Ed Hardy Womens NYC Long Sleeve Tee Shirt-Black,17,Tops & Tees,Women
1,7689,Sutton Studio Velvet Double Breast Jacket,17,Blazers & Jackets,Women
2,175,Woman Within Plus Size Top sweatshirt in cozy ...,16,Tops & Tees,Women
3,6916,Volcom Juniors High Tower Short,16,Shorts,Women
4,11129,Flexees Women's Everyday Control Lace Insert B...,16,Intimates,Women
5,1494,Joie Women's Lucero,16,Sweaters,Women
6,10943,Hanes Women's Classic Cotton 3-Pack Low Rise B...,15,Intimates,Women
7,9888,Pretty Caftan with Midnight Floral Vines Up2da...,15,Sleep & Lounge,Women
8,11058,ASICS Women's Stride Running Brief,15,Intimates,Women
9,7760,Plus Size Black Jazzy Jacket,15,Blazers & Jackets,Women
